In [2]:
import ipy_table
from collections import Counter
from IPython.display import HTML, display
import tabulate
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

%run ../aux/postgres.py

postgres functions loaded...


In [151]:
cmd = "select count(codtweet) from amostratweet where codamostra = 105"
res = query(cmd)
print(res)

cmd = ("select * from analisetweet where codanaliseamostra = 127 "
       "and codtweet not in (select codtweet from amostratweet where codamostra = 105)")

res = query(cmd)
print(len(res))
print(res[:10])

[(256,)]
33
[(307833, 127, 232, 148874837, None), (307834, 127, 234, 148874837, None), (307846, 127, 233, 151343640, None), (307851, 127, 233, 152032719, None), (307879, 127, 233, 152571746, None), (307852, 127, 233, 152762460, None), (307899, 127, 233, 152774202, None), (307853, 127, 233, 152963729, None), (307884, 127, 233, 156100029, None), (307906, 127, 232, 156812873, None)]


In [18]:
#query("delete from analisetweet where codanaliseamostra = 127 and codtweet not in (select codtweet from amostratweet where codamostra = 105)")

In [16]:
query("select count(codtweet) from amostratweet where codamostra = 105")

[(256,)]

In [197]:
cmd = ("SELECT am.codanaliseamostra, codamostra, hash, p.nome, "
       "(select count(distinct codtweet) from amostratweet where codamostra = am.codamostra) AS avaliar , "
       "(select count(distinct codtweet) from analisetweet where codanaliseamostra = am.codanaliseamostra) AS avaliados  "
       "FROM analiseamostra am "
       "INNER JOIN usuario p ON am.codusuario = p.codusuario "
       "WHERE hash like 'fake%' "
       "ORDER BY codamostra, p.nome")
res = query(cmd)
res.insert(0,('cod','amostra','hash','avaliador','avaliar','avaliados'))
ipy_table.make_table(res)
ipy_table.set_global_style(wrap=True)
ipy_table.apply_theme('basic')

cod,amostra,hash,avaliador,avaliar,avaliados
128,105,fake141601,Maria Eduarda Sampaio de Sousa,256,256
127,105,fake371101,Mateus Tarcinalli Machado,256,256
130,107,fake371102,Maria Eduarda Sampaio de Sousa,8,0
148,107,fake364779,Mateus Tarcinalli Machado,8,8
131,109,fake171801,Maria Eduarda Sampaio de Sousa,3,3
145,109,fake171889,Mateus Tarcinalli Machado,3,3
132,111,fake181702,Maria Eduarda Sampaio de Sousa,181,181
146,111,fake188963,Mateus Tarcinalli Machado,181,181
133,113,fake222306,Maria Eduarda Sampaio de Sousa,50,50
144,113,fake456896,Mateus Tarcinalli Machado,50,50


In [152]:
cmd = ("SELECT am.codanaliseamostra, codamostra, hash, p.nome, "
       "(select count(distinct codtweet) from amostratweet where codamostra = am.codamostra) AS avaliar , "
       "(select count(distinct codtweet) from analisetweet where codanaliseamostra = am.codanaliseamostra) AS avaliados  "
       "FROM analiseamostra am "
       "INNER JOIN usuario p ON am.codusuario = p.codusuario "
       "WHERE hash like 'fake%' and p.nome like 'Maria%'"
       "ORDER BY codamostra, p.nome")
res = query(cmd)
res.insert(0,('cod','amostra','hash','avaliador','avaliar','avaliados'))
ipy_table.make_table(res)
ipy_table.set_global_style(wrap=True)
ipy_table.apply_theme('basic')

cod,amostra,hash,avaliador,avaliar,avaliados
128,105,fake141601,Maria Eduarda Sampaio de Sousa,256,105
130,107,fake371102,Maria Eduarda Sampaio de Sousa,8,0
131,109,fake171801,Maria Eduarda Sampaio de Sousa,3,3
132,111,fake181702,Maria Eduarda Sampaio de Sousa,181,0
133,113,fake222306,Maria Eduarda Sampaio de Sousa,50,0
136,115,fake171903,Maria Eduarda Sampaio de Sousa,12682,101
137,117,fake156789,Maria Eduarda Sampaio de Sousa,52,0
138,119,fake136859,Maria Eduarda Sampaio de Sousa,1128,0
139,121,fake213678,Maria Eduarda Sampaio de Sousa,223,0
140,123,fake148596,Maria Eduarda Sampaio de Sousa,39,39


In [43]:
def matrixConfusao(av01, av02, labels, labelsN):
    a = confusion_matrix(av01, av02, labels=labels).tolist()
    a.insert(0,labelsN)
    labelsNames = [''] + labelsN
    b = []
    i = 0
    for x in a:
        x = list(x)
        x.insert(0,(labelsNames[i]))
        x = tuple(x)
        i = i + 1
        b.append(x)
    #print(b)
    return(b)
    #ipy_table.render()

## Noticia 01

In [198]:
# Eduarda
cmd = ("SELECT cl.descricao "
       "FROM analisetweet at "
       "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
       "WHERE codanaliseamostra = 128 and cl.codclassificacao <> 36"
       "ORDER BY cl.descricao"
      )
res01 = query(cmd)
topics = [x[0] for x in res01]
count01 = Counter(topics)
ipy_table.make_table(count01.most_common())

Não Relacionado,231
Propagação de Fake News,59
Relacionado,25
Contra Fake News,15
Réplica,2


In [199]:
# Mateus
cmd = ("SELECT cl.descricao "
       "FROM analisetweet at "
       "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
       "WHERE codanaliseamostra = 127 and cl.codclassificacao <> 36"
       "ORDER BY cl.descricao"
      )
res01 = query(cmd)
topics = [x[0] for x in res01]
count01 = Counter(topics)
ipy_table.make_table(count01.most_common())

Não Relacionado,229
Relacionado,27
Contra Fake News,15
Propagação de Fake News,8
Réplica,4


In [204]:
# kapa entre avaliação 128 (eduarda) e 127 (mateus)

# carregando avaliação mateus
cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
       "FROM analisetweet at "
       "WHERE codanaliseamostra = 128 "
       "AND codclassificacao in (233, 232) "
       "AND codtweet in (select codtweet from analisetweet where codanaliseamostra = 127) "
       "order by codtweet"
      )
res = query(cmd)
avMateus = [ x[1] for x in res ]

# carregando avaliação eduarda
cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
       "FROM analisetweet at "
       "WHERE codanaliseamostra = 127 "
       "AND codclassificacao in (233, 232) "
       "and codtweet in (select codtweet from analisetweet where codanaliseamostra = 128) "
       "and at.codclassificacao <> 36  order by codtweet"
      )
res = query(cmd)
avEduarda = [ x[1] for x in res ]

print("Eduarda e Mateus: %s" % cohen_kappa_score(avEduarda, avMateus))

Eduarda e Mateus: 0.9571977930112021


In [205]:
ipy_table.make_table(matrixConfusao(avEduarda, avMateus, [232,233], ['relacionado','não relacionado']))

,relacionado,não relacionado
relacionado,25,2
não relacionado,0,229


In [206]:
# kapa entre avaliação 128 (eduarda) e 127 (mateus)

# carregando avaliação mateus
cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
       "FROM analisetweet at "
       "WHERE codanaliseamostra = 128 "
       "AND codclassificacao in (234, 235, 236) "
       "AND codtweet in (select codtweet from analisetweet where codanaliseamostra = 127 AND codclassificacao in (234, 235, 236)) "
       "order by codtweet"
      )
res = query(cmd)
avMateus = [ x[1] for x in res ]

# carregando avaliação eduarda
cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
       "FROM analisetweet at "
       "WHERE codanaliseamostra = 127 "
       "AND codclassificacao in (234, 235, 236) "
       "and codtweet in (select codtweet from analisetweet where codanaliseamostra = 128 AND codclassificacao in (234, 235, 236)) "
       "and at.codclassificacao <> 36  order by codtweet"
      )
res = query(cmd)
avEduarda = [ x[1] for x in res ]

print("Eduarda e Mateus: %s" % cohen_kappa_score(avEduarda, avMateus))

Eduarda e Mateus: 0.6585365853658537


In [207]:
ipy_table.make_table(matrixConfusao(avEduarda, avMateus, [234,235,236], ['fake','contra','replica']))

,fake,contra,replica
fake,5,1,1
contra,0,11,0
replica,0,2,1


In [208]:
compara(133, 144, "Eduarda", "Mateus")

In [209]:
def compara(codanaliseamostra1, codanaliseamostra2, av1, av2):
    codanaliseamostra1 = str(codanaliseamostra1)
    codanaliseamostra2 = str(codanaliseamostra2)
    # av 01
    cmd = ("SELECT cl.descricao "
           "FROM analisetweet at "
           "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
           "WHERE codanaliseamostra = " + codanaliseamostra1 + " and cl.codclassificacao <> 36 "
           "ORDER BY cl.descricao"
          )
    res01 = query(cmd)
    topics = [x[0] for x in res01]
    count01 = Counter(topics)
    tbl01 = ipy_table.make_table(count01.most_common())
        
    # av 02
    cmd = ("SELECT cl.descricao "
           "FROM analisetweet at "
           "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
           "WHERE codanaliseamostra = " + codanaliseamostra2 + " and cl.codclassificacao <> 36 "
           "ORDER BY cl.descricao"
          )
    res01 = query(cmd)
    topics = [x[0] for x in res01]
    count01 = Counter(topics)
    tbl02 = ipy_table.make_table(count01.most_common())
    
    ret = "<table><tr><td>" + av1 + ":" 
    ret = ret + tbl01._repr_html_() + '</td><td>' + av2 + ":"
    ret = ret + tbl02._repr_html_() + "</td></tr></table>"
    
    # carregando avaliação 01
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra1 + " "
           "AND codclassificacao in (233, 232) "
           "AND codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra2 + ") "
           "order by codtweet"
          )
    res = query(cmd)
    avMateus = [ x[1] for x in res ]

    # carregando avaliação 02
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra2 + " "
           "AND codclassificacao in (233, 232) "
           "and codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra1 + ") "
           "and at.codclassificacao <> 36  order by codtweet"
          )
    res = query(cmd)
    avEduarda = [ x[1] for x in res ]

    ret = ret + "<h2>Eixo 01: Relação com a notícia</h2>"
    ret = ret + ("<br>Kappa: " + av1 + " X " + av2 + ": %s" % cohen_kappa_score(avEduarda, avMateus))    
    tbl = ipy_table.make_table(matrixConfusao(avEduarda, avMateus, [232,233], ['relacionado','não relacionado']))
    ret = ret + tbl._repr_html_()
    
    
    # eixo 2

    # carregando avaliação mateus
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra1 + " "
           "AND codclassificacao in (234, 235, 236) "
           "AND codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra2 + " AND codclassificacao in (234, 235, 236)) "
           "order by codtweet"
          )
    res = query(cmd)
    avMateus = [ x[1] for x in res ]

    # carregando avaliação eduarda
    cmd = ("SELECT distinct on (at.codtweet) at.codtweet , at.codclassificacao "
           "FROM analisetweet at "
           "WHERE codanaliseamostra = " + codanaliseamostra2 + " "
           "AND codclassificacao in (234, 235, 236) "
           "and codtweet in (select codtweet from analisetweet "
           "where codanaliseamostra = " + codanaliseamostra1 + " AND codclassificacao in (234, 235, 236)) "
           "and at.codclassificacao <> 36  order by codtweet"
          )
    res = query(cmd)
    avEduarda = [ x[1] for x in res ]

    ret = ret + "<h2>Eixo 02: Fake News</h2>"
    ret = ret + ("<br>Kappa: " + av1 + " X " + av2 + ": %s" % cohen_kappa_score(avEduarda, avMateus))    
    #print("Eduarda e Mateus: %s" % cohen_kappa_score(avEduarda, avMateus))    
    
    tbl = ipy_table.make_table(matrixConfusao(avEduarda, avMateus, [234,235,236], ['Propagação','Contra','Réplica']))
    ret = ret + tbl._repr_html_()
    
    
    return HTML(ret)

In [224]:
cmd = ("SELECT am.codanaliseamostra, codamostra, hash, p.nome, "
       "(select count(distinct codtweet) from amostratweet where codamostra = am.codamostra) AS avaliar , "
       "(select count(distinct codtweet) from analisetweet where codanaliseamostra = am.codanaliseamostra) AS avaliados  "
       "FROM analiseamostra am "
       "INNER JOIN usuario p ON am.codusuario = p.codusuario "
       "WHERE hash like 'fake%' and p.nome like 'Mateus%'"
       "ORDER BY codamostra, p.nome")
res = query(cmd)
res.insert(0,('cod','amostra','hash','avaliador','avaliar','avaliados'))
ipy_table.make_table(res)
ipy_table.set_global_style(wrap=True)
ipy_table.apply_theme('basic')

cod,amostra,hash,avaliador,avaliar,avaliados
127,105,fake371101,Mateus Tarcinalli Machado,256,256
148,107,fake364779,Mateus Tarcinalli Machado,8,8
145,109,fake171889,Mateus Tarcinalli Machado,3,3
146,111,fake188963,Mateus Tarcinalli Machado,181,181
144,113,fake456896,Mateus Tarcinalli Machado,50,50
149,115,fake963478,Mateus Tarcinalli Machado,12682,40
150,117,fake199873,Mateus Tarcinalli Machado,52,52
147,119,fake196347,Mateus Tarcinalli Machado,1128,20
151,121,fake296347,Mateus Tarcinalli Machado,223,128
152,123,fake897426,Mateus Tarcinalli Machado,39,39


In [225]:
cmd = ("SELECT am.codanaliseamostra, a.descricao as amostra, hash, p.nome, "
       "(select count(distinct codtweet) from amostratweet where codamostra = am.codamostra) AS avaliar , "
       "(select count(distinct codtweet) from analisetweet where codanaliseamostra = am.codanaliseamostra) AS avaliados  "
       "FROM analiseamostra am "
       "INNER JOIN usuario p ON am.codusuario = p.codusuario "
       "INNER JOIN amostra a on am.codamostra = a.codamostra "
       "WHERE hash like 'fake%' "
       "ORDER BY am.codamostra, p.nome")
res = query(cmd)
res.insert(0,('cod','amostra','hash','avaliador','avaliar','avaliados'))
ipy_table.make_table(res)
ipy_table.set_global_style(wrap=True)
ipy_table.apply_theme('basic')

cod,amostra,hash,avaliador,avaliar,avaliados
128,Eleições Notícia 01 [no similar],fake141601,Maria Eduarda Sampaio de Sousa,256,256
127,Eleições Notícia 01 [no similar],fake371101,Mateus Tarcinalli Machado,256,256
130,Eleições Notícia 02 [no similar],fake371102,Maria Eduarda Sampaio de Sousa,8,0
148,Eleições Notícia 02 [no similar],fake364779,Mateus Tarcinalli Machado,8,8
131,Eleições Notícia 03 [no similar],fake171801,Maria Eduarda Sampaio de Sousa,3,3
145,Eleições Notícia 03 [no similar],fake171889,Mateus Tarcinalli Machado,3,3
132,Eleições Notícia 04 [no similar],fake181702,Maria Eduarda Sampaio de Sousa,181,181
146,Eleições Notícia 04 [no similar],fake188963,Mateus Tarcinalli Machado,181,181
133,Eleições Notícia 05 [no similar],fake222306,Maria Eduarda Sampaio de Sousa,50,50
144,Eleições Notícia 05 [no similar],fake456896,Mateus Tarcinalli Machado,50,50


## Notícia 01

In [226]:
compara(128, 127, "Eduarda", "Mateus")

## Notícia 02

In [227]:
compara(130, 148, "Eduarda", "Mateus")

IndexError: list index out of range

## Notícia 03

In [228]:
compara(131, 145, "Eduarda", "Mateus")

/home/mateus/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:576: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


## Notícia 4

In [229]:
compara(132, 146, "Eduarda", "Mateus")

## Notícia 05

In [230]:
compara(133, 144, "Eduarda", "Mateus")

## Notícia 06

In [231]:
compara(136, 149, "Eduarda", "Mateus")

## Notícia 07

In [232]:
compara(137, 150, "Eduarda", "Mateus")

## Notícia 08

In [233]:
compara(138, 147, "Eduarda", "Mateus")

## Notícia 09

In [234]:
compara(139, 151, "Eduarda", "Mateus")

## Notícia 10

In [235]:
compara(140, 152, "Eduarda", "Mateus")

## Notícia 14

In [236]:
compara(141, 153, "Eduarda", "Mateus")

/home/mateus/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:576: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


## Notícia 15

In [237]:
compara(142, 154, "Eduarda", "Mateus")